In [23]:
import importlib
import pandas as pd
import YandexGPT_API
importlib.reload(YandexGPT_API)
import chromadb
from chromadb.config import Settings
import json

In [2]:
df_news = pd.read_csv('news_cut.csv').head(50)
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     50 non-null     object
 1   title   50 non-null     object
 2   text    50 non-null     object
 3   topic   47 non-null     object
 4   tags    49 non-null     object
 5   date    50 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [3]:
df_news.drop(['title', 'topic', 'tags', 'date'], axis=1, inplace=True)

In [4]:
df_news.head(5)

,url,text
0,https://lenta.ru/news/2006/06/25/marketreview/,"Индекс РТС на торгах 23 июня опустился на 0,68..."
1,https://lenta.ru/news/2001/09/04/ntv/,"Бизнесмен и солист группы ""Белый орел"" Владими..."
2,https://lenta.ru/news/2009/03/30/monsters/,"Анимационный фильм ""Монстры против пришельцев""..."
3,https://lenta.ru/news/2018/01/04/valdes/,Чемпион мира и Европы в составе сборной Испани...
4,https://lenta.ru/news/2016/08/25/ninaeremina/,Чемпионка мира по баскетболу в составе сборной...


In [5]:
# message = [{
#     "role": "user",
#     "text": "Привет, давай знакомиться"
# }]
# answer = gpt.make_request(message)
# pprint(answer)

In [9]:
gpt = YandexGPT_API.YandexGPTEmbeddings()

In [6]:
message = 'Кто проживает на дне океана'
answer = gpt.embed_document(message)

<Response [200]>
{'Authorization': 'Api-key AQVNzbEhq00ywPjhTYnHRuHdbncqpvW7aRKhyWAv', 'x-folder-id': 'b1ggqo8gtd5csohggehu'}


In [11]:
answers = gpt.embed_documents(df_news['text'])

100%|██████████| 50/50 [00:54<00:00,  1.08s/it]


In [12]:
answers

[[0.10400390625,
  0.0738525390625,
  0.037200927734375,
  -0.0267791748046875,
  0.00559234619140625,
  -0.0673828125,
  0.041015625,
  -0.052947998046875,
  0.0703125,
  -0.1162109375,
  0.07061767578125,
  -0.061187744140625,
  0.0238800048828125,
  0.037933349609375,
  0.12249755859375,
  -0.0592041015625,
  0.0318603515625,
  -0.0259246826171875,
  -0.0258026123046875,
  -0.06170654296875,
  0.028228759765625,
  -0.07061767578125,
  -0.06414794921875,
  0.0185699462890625,
  -0.10986328125,
  -0.048553466796875,
  0.00524139404296875,
  -0.08843994140625,
  0.06915283203125,
  -0.0033111572265625,
  -0.0494384765625,
  0.01983642578125,
  0.0247802734375,
  -0.08380126953125,
  -0.005535125732421875,
  -0.07269287109375,
  -0.049285888671875,
  0.01739501953125,
  0.089111328125,
  -0.068115234375,
  -0.00826263427734375,
  -0.01358795166015625,
  0.0052490234375,
  0.012420654296875,
  0.059539794921875,
  0.0875244140625,
  -0.01068115234375,
  0.01399993896484375,
  -0.07263183

In [14]:
df_news['embeds'] = answers
df_news.to_csv('news_embed.csv', index=False)

In [31]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))

collection = chroma_client.get_or_create_collection("news")

In [32]:
df = pd.read_csv('news_embed.csv')
texts = df["text"].tolist()

In [33]:
text_embeddings =  list(map(json.loads, df["embeds"].tolist()))
ids = df_news.index.astype(str).tolist()

In [34]:
collection.upsert(
    ids=ids,
    embeddings=text_embeddings,
    metadatas=[{"source": "news", "text": txt} for txt in texts],
    documents=texts
)